In [ ]:
import aiohttp


async def fetch_pvis_data(session: aiohttp.ClientSession, proceeding_id: str):
    url = "https://www.computer.org/csdl/api/v1/graphql"
    USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0.1 Safari/605.1.15"
    headers = {"content-type": "application/json", "User-Agent": USER_AGENT}
    payload = {
        "query": """query ($proceedingId: String!, $limitResults: Int, $skipResults: Int) {
  proceeding: proceeding(proceedingId: $proceedingId) {
    id
    groupId
    acronym
    issn
    isbn
    startDate
    endDate
    location
    website
    title
    volume
    displayVolume
    year
    __typename
  }
  articlesByProceeding: articlesByProceedingWithPagination(
    proceedingId: $proceedingId
    limit: $limitResults
    skip: $skipResults
  ) {
    limit
    skipped
    totalResults
    articleResults {
      id
      pubType
      pubDate
      doi
      idPrefix
      authors {
        fullName
        givenName
        surname
        __typename
      }
      fno
      isOpenAccess
      issueNum
      pages
      sectionTitle
      title
      year
      __typename
    }
    __typename
  }
  proceedingVolumes: proceedingVolumes(proceedingId: $proceedingId) {
    id
    volume
    __typename
  }
}
""",
        "variables": {
            "proceedingId": proceeding_id,
            "limitResults": 200,
            "skipResults": 0,
        },
    }

    async with session.post(url, json=payload, headers=headers) as response:
        data = await response.json()

    return data

In [4]:
pvis_ids = {
    "2015": "12OmNAQJzKb",
    "2016": "12OmNxTEiSt",
    "2017": "12OmNxdVh2r",
    "2018": "12OmNykCcdo",
    "2019": "1cpqjBXCukg",
    "2020": "1kmoNreKiTm",
    "2021": "1tTtoVK3gYg",
    "2022": "1E2weOclERO",
    "2023": "1NYhp1c4BxK",
    "2024": "1Xtul825y9i",
    "2025": "27ASaQI6DbG",
}

In [3]:
async with aiohttp.ClientSession() as session:
    data = await fetch_pvis_data(session, "27ASaQI6DbG")
    print(data)

{'data': {'proceeding': {'id': '27ASaQI6DbG', 'groupId': '1001657', 'acronym': 'pacificvis', 'issn': None, 'isbn': '979-8-3315-0581-3', 'startDate': 'April 22 2025', 'endDate': 'April 25 2025', 'location': 'Taipei City, Taiwan', 'website': None, 'title': '2025 IEEE 18th Pacific Visualization Conference (PacificVis)', 'volume': '0', 'displayVolume': '0', 'year': '2025', '__typename': 'ProceedingType'}, 'articlesByProceeding': {'limit': 200, 'skipped': 0, 'totalResults': 50, 'articleResults': [{'id': '27ASe3XWKdy', 'pubType': 'proceedings', 'pubDate': '2025-04-01', 'doi': '10.1109/PacificVis64226.2025.00001', 'idPrefix': 'pacificvis', 'authors': [], 'fno': '058100z001', 'isOpenAccess': True, 'issueNum': None, 'pages': '1-1', 'sectionTitle': None, 'title': 'Title Page i', 'year': '2025', '__typename': 'ArticleType'}, {'id': '27ASgolfb44', 'pubType': 'proceedings', 'pubDate': '2025-04-01', 'doi': '10.1109/PacificVis64226.2025.00002', 'idPrefix': 'pacificvis', 'authors': [], 'fno': '058100z

In [14]:
papers = []
async with aiohttp.ClientSession() as session:
    for year, proceeding_id in pvis_ids.items():
        data = await fetch_pvis_data(session, proceeding_id)
        p = []
        for article in data['data']['articlesByProceeding']['articleResults']:
            is_paper = not article["isOpenAccess"]
            if is_paper:
                p.append(article)
        print(f"Year {year} has {len(p)} papers")
        papers.extend(p)

print(len(papers))

Year 2015 has 45 papers
Year 2016 has 39 papers
Year 2017 has 38 papers
Year 2018 has 30 papers
Year 2019 has 32 papers
Year 2020 has 30 papers
Year 2021 has 30 papers
Year 2022 has 24 papers
Year 2023 has 26 papers
Year 2024 has 43 papers
Year 2025 has 45 papers
382
